# Day 17: Call Options Deep Dive

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week4_options_basics/day17_call_options.ipynb)

---

## Learning Objectives

1. **Master** call option mechanics and payoffs
2. **Understand** when to buy vs sell calls
3. **Learn** the covered call strategy
4. **Calculate** breakeven, max profit, and max loss
5. **Apply** call options to real scenarios

---

## Lecture (30 minutes)

### Call Option Review

```
CALL OPTION FUNDAMENTALS
========================

CALL = Right to BUY stock at strike price

BUYER (Long Call):
- Pays premium
- Wants stock to go UP
- Limited risk (premium)
- Unlimited profit potential

SELLER (Short Call):
- Receives premium
- Wants stock to stay flat or go DOWN
- Limited profit (premium)
- Unlimited risk (unless covered)
```

### Buying Calls (Long Calls)

```
LONG CALL STRATEGY
==================

OUTLOOK: Bullish (expect stock to rise)

SETUP:
Buy 1 Call at Strike K for Premium P

KEY NUMBERS:
Max Loss = Premium paid (P)
Breakeven = Strike + Premium (K + P)
Max Profit = Unlimited

EXAMPLE:
Stock at $175, Buy $180 Call for $3.00

Max Loss = $3.00 × 100 = $300
Breakeven = $180 + $3 = $183
Max Profit = Unlimited

AT EXPIRATION:
Stock at $190: Profit = ($190 - $183) × 100 = $700
Stock at $180: Loss = $300 (expires worthless)
Stock at $170: Loss = $300 (expires worthless)

LEVERAGE:
Control $18,000 of stock for $300
60:1 leverage ratio!
```

### When to Buy Calls

```
LONG CALL DECISION FRAMEWORK
============================

BUY CALLS WHEN:
1. Strongly bullish on stock
2. Want leverage on upside
3. Defined max loss is important
4. Expect move BEFORE expiration
5. Implied volatility is LOW (options cheap)

DON'T BUY CALLS WHEN:
1. Moderately bullish (use spreads)
2. Expect slow, gradual rise
3. IV is high (options expensive)
4. No clear catalyst
5. Can't afford total loss

STRIKE SELECTION:
ITM: Lower risk, lower return, higher cost
ATM: Balanced risk/reward
OTM: Higher risk, higher return, lower cost

EXPIRATION SELECTION:
- Give yourself TIME to be right
- Avoid weeklies for beginners
- 30-60 days minimum
- Consider LEAPS for long-term plays
```

### Covered Calls

```
COVERED CALL STRATEGY
=====================

SETUP:
Own 100 shares of stock
+ Sell 1 Call option

"COVERED" = You own the shares to deliver

OUTLOOK: Neutral to moderately bullish

EXAMPLE:
Own 100 shares at $175
Sell $185 Call for $2.50

SCENARIOS AT EXPIRATION:

1. Stock at $170 (below strike):
   - Keep shares, keep premium
   - Stock loss: -$500
   - Premium gained: +$250
   - Net: -$250 (better than -$500)

2. Stock at $180 (below strike):
   - Keep shares, keep premium
   - Stock gain: +$500
   - Premium: +$250
   - Net: +$750

3. Stock at $190 (above strike):
   - Shares called away at $185
   - Stock gain: +$1,000
   - Premium: +$250
   - But miss $500 above $185
   - Net: +$1,250 (capped)

MAX PROFIT = (Strike - Cost Basis) + Premium
MAX LOSS = Cost Basis - Premium (stock to $0)
BREAKEVEN = Cost Basis - Premium
```

### Covered Call Mechanics

```
COVERED CALL TRADE MANAGEMENT
=============================

OPENING THE TRADE:
1. Own or buy 100 shares
2. Sell OTM call (5-10% above)
3. Collect premium immediately

STRIKE SELECTION:
- Too close: High premium, likely called away
- Too far: Low premium, minimal income
- Sweet spot: 5-10% OTM, 1-2% monthly yield

EXPIRATION SELECTION:
- 30-45 days optimal (best theta decay)
- Monthly options most liquid
- Avoid earnings if you want to keep shares

MANAGEMENT:
If stock rises (option ITM):
- Let shares get called away
- Or roll up and out (buy back, sell higher)

If stock falls:
- Keep premium (cushions loss)
- Sell another call next month

If stock flat:
- Keep premium, keep shares
- Repeat monthly = income stream
```

### Call Selling Risks

```
NAKED vs COVERED CALLS
======================

COVERED CALL (Safe):
- Own the shares
- If assigned, deliver your shares
- Max loss = stock going to $0
- No margin requirement

NAKED CALL (Dangerous!):
- DON'T own the shares
- If assigned, must buy at market to deliver
- UNLIMITED loss potential
- High margin requirement
- NOT for beginners!

EXAMPLE OF NAKED CALL DISASTER:
Sell $50 Call for $2 (no shares)
Stock squeezes to $150
Loss = ($150 - $50 - $2) × 100 = $9,800 per contract!

RULE: Never sell naked calls as a beginner
Always have the shares (covered) or a long call (spread)
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def long_call_calculator(stock_price, strike, premium, contracts=1):
    """Calculate long call trade metrics."""
    shares = contracts * 100
    total_cost = premium * shares
    breakeven = strike + premium
    
    print(f"\n{'='*50}")
    print("LONG CALL CALCULATOR")
    print(f"{'='*50}")
    print(f"\nCurrent Stock Price: ${stock_price:.2f}")
    print(f"Strike Price: ${strike:.2f}")
    print(f"Premium: ${premium:.2f} per share")
    print(f"Contracts: {contracts} ({shares} shares)")
    
    print(f"\n{'-'*50}")
    print("TRADE METRICS:")
    print(f"{'-'*50}")
    print(f"Total Cost: ${total_cost:,.2f}")
    print(f"Breakeven: ${breakeven:.2f}")
    print(f"Max Loss: ${total_cost:,.2f} (100% of investment)")
    print(f"Max Profit: Unlimited")
    
    # Required move
    move_to_be = (breakeven - stock_price) / stock_price * 100
    print(f"\nRequired Move to Breakeven: {move_to_be:+.1f}%")
    
    # Leverage
    stock_value = strike * shares
    leverage = stock_value / total_cost
    print(f"Effective Leverage: {leverage:.1f}x")
    
    return breakeven, total_cost

# Example long call
be, cost = long_call_calculator(stock_price=175, strike=180, premium=4.50, contracts=2)

In [ ]:
def plot_long_call_payoff(strike, premium, stock_range=None):
    """Plot long call payoff with detailed annotations."""
    if stock_range is None:
        stock_range = np.linspace(strike * 0.7, strike * 1.4, 100)
    
    # Calculate payoff
    payoff = np.maximum(stock_range - strike, 0) - premium
    breakeven = strike + premium
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot payoff line
    ax.plot(stock_range, payoff, 'b-', lw=2.5, label='Long Call Payoff')
    
    # Fill areas
    ax.fill_between(stock_range, payoff, 0, where=payoff > 0, alpha=0.3, color='green', label='Profit Zone')
    ax.fill_between(stock_range, payoff, 0, where=payoff < 0, alpha=0.3, color='red', label='Loss Zone')
    
    # Reference lines
    ax.axhline(y=0, color='black', linestyle='-', lw=1)
    ax.axhline(y=-premium, color='red', linestyle='--', alpha=0.5, label=f'Max Loss: ${premium}')
    ax.axvline(x=strike, color='gray', linestyle='--', alpha=0.5, label=f'Strike: ${strike}')
    ax.axvline(x=breakeven, color='green', linestyle='--', alpha=0.5, label=f'Breakeven: ${breakeven}')
    
    # Annotations
    ax.annotate(f'Max Loss: -${premium}', xy=(strike * 0.75, -premium), fontsize=11, color='red')
    ax.annotate(f'Strike: ${strike}', xy=(strike, premium), fontsize=10)
    ax.annotate(f'Breakeven: ${breakeven}', xy=(breakeven + 2, 1), fontsize=10, color='green')
    
    ax.set_xlabel('Stock Price at Expiration', fontsize=12)
    ax.set_ylabel('Profit/Loss per Share', fontsize=12)
    ax.set_title(f'Long Call Payoff Diagram\nStrike: ${strike} | Premium: ${premium}', fontsize=14)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_long_call_payoff(strike=180, premium=4.50)

In [ ]:
def covered_call_calculator(shares_cost_basis, current_price, strike, premium, shares=100):
    """Calculate covered call trade metrics."""
    contracts = shares // 100
    total_premium = premium * shares
    
    print(f"\n{'='*60}")
    print("COVERED CALL CALCULATOR")
    print(f"{'='*60}")
    print(f"\nShares Owned: {shares} (Cost Basis: ${shares_cost_basis:.2f})")
    print(f"Current Stock Price: ${current_price:.2f}")
    print(f"Call Strike: ${strike:.2f}")
    print(f"Premium Received: ${premium:.2f} per share")
    
    # Calculations
    breakeven = shares_cost_basis - premium
    max_profit = (strike - shares_cost_basis + premium) * shares
    max_profit_pct = max_profit / (shares_cost_basis * shares) * 100
    
    # If assigned
    if_called = (strike - shares_cost_basis + premium) * shares
    
    # Yield metrics
    monthly_yield = (premium / current_price) * 100
    annual_yield = monthly_yield * 12
    
    print(f"\n{'-'*60}")
    print("TRADE METRICS:")
    print(f"{'-'*60}")
    print(f"Total Premium Collected: ${total_premium:,.2f}")
    print(f"Breakeven Price: ${breakeven:.2f}")
    print(f"Max Profit (if called): ${max_profit:,.2f} ({max_profit_pct:.1f}%)")
    print(f"Max Loss (stock to $0): ${(shares_cost_basis - premium) * shares:,.2f}")
    
    print(f"\n{'-'*60}")
    print("INCOME ANALYSIS:")
    print(f"{'-'*60}")
    print(f"Premium Yield (this month): {monthly_yield:.2f}%")
    print(f"Annualized Yield: {annual_yield:.1f}%")
    
    # Scenarios
    print(f"\n{'-'*60}")
    print("SCENARIOS AT EXPIRATION:")
    print(f"{'-'*60}")
    
    scenarios = [current_price * 0.9, current_price, strike, strike * 1.05]
    
    for price in scenarios:
        stock_pnl = (price - shares_cost_basis) * shares
        
        if price >= strike:
            # Called away
            total_pnl = (strike - shares_cost_basis + premium) * shares
            result = "CALLED AWAY"
        else:
            # Keep shares
            total_pnl = stock_pnl + total_premium
            result = "KEEP SHARES"
        
        print(f"\nStock at ${price:.2f}: {result}")
        print(f"  Stock P/L: ${stock_pnl:+,.2f}")
        print(f"  Premium: +${total_premium:,.2f}")
        print(f"  Total P/L: ${total_pnl:+,.2f}")
    
    return breakeven, max_profit, total_premium

# Example covered call
covered_call_calculator(
    shares_cost_basis=170, 
    current_price=175, 
    strike=185, 
    premium=2.50
)

In [ ]:
def plot_covered_call_payoff(cost_basis, strike, premium):
    """Plot covered call vs stock only comparison."""
    stock_range = np.linspace(cost_basis * 0.7, strike * 1.2, 100)
    
    # Stock only P/L
    stock_pnl = stock_range - cost_basis
    
    # Covered call P/L
    covered_pnl = np.where(
        stock_range >= strike,
        strike - cost_basis + premium,  # Capped if called
        stock_range - cost_basis + premium  # Stock gain + premium
    )
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot both
    ax.plot(stock_range, stock_pnl, 'b--', lw=2, label='Stock Only', alpha=0.7)
    ax.plot(stock_range, covered_pnl, 'g-', lw=2.5, label='Covered Call')
    
    # Reference lines
    ax.axhline(y=0, color='black', linestyle='-', lw=1)
    ax.axvline(x=cost_basis, color='gray', linestyle=':', alpha=0.5, label=f'Cost Basis: ${cost_basis}')
    ax.axvline(x=strike, color='red', linestyle='--', alpha=0.5, label=f'Strike: ${strike}')
    
    # Shade regions
    ax.fill_between(stock_range, covered_pnl, stock_pnl, 
                    where=covered_pnl > stock_pnl, alpha=0.2, color='green', 
                    label='Covered Call Advantage')
    ax.fill_between(stock_range, covered_pnl, stock_pnl, 
                    where=covered_pnl < stock_pnl, alpha=0.2, color='red', 
                    label='Stock Only Advantage')
    
    # Annotations
    max_profit = strike - cost_basis + premium
    ax.annotate(f'Max Profit: ${max_profit:.2f}', 
                xy=(strike * 1.1, max_profit), fontsize=10, color='green')
    ax.annotate(f'Premium cushion: ${premium:.2f}', 
                xy=(cost_basis * 0.85, premium/2), fontsize=10)
    
    ax.set_xlabel('Stock Price at Expiration', fontsize=12)
    ax.set_ylabel('Profit/Loss per Share', fontsize=12)
    ax.set_title(f'Covered Call vs Stock Only\nCost: ${cost_basis} | Strike: ${strike} | Premium: ${premium}', fontsize=14)
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_covered_call_payoff(cost_basis=170, strike=185, premium=2.50)

In [ ]:
def find_covered_call_opportunities(ticker):
    """Find attractive covered call opportunities."""
    stock = yf.Ticker(ticker)
    info = stock.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    expirations = stock.options
    
    if len(expirations) == 0:
        print(f"No options available for {ticker}")
        return
    
    print(f"\n{'='*70}")
    print(f"COVERED CALL OPPORTUNITIES: {ticker}")
    print(f"{'='*70}")
    print(f"\nCurrent Price: ${current_price:.2f}\n")
    
    # Analyze first 3 expirations
    for exp in expirations[:3]:
        try:
            options = stock.option_chain(exp)
            calls = options.calls
            
            # Filter OTM calls (5-15% above current price)
            otm_calls = calls[
                (calls['strike'] > current_price * 1.03) & 
                (calls['strike'] < current_price * 1.15) &
                (calls['bid'] > 0)
            ].copy()
            
            if len(otm_calls) == 0:
                continue
            
            print(f"{'-'*70}")
            print(f"Expiration: {exp}")
            print(f"{'-'*70}")
            
            for _, row in otm_calls.head(4).iterrows():
                strike = row['strike']
                premium = row['bid']  # Use bid for realistic fill
                
                # Calculate metrics
                upside = (strike - current_price) / current_price * 100
                premium_yield = premium / current_price * 100
                
                print(f"\n  Strike: ${strike:.2f} ({upside:+.1f}% upside)")
                print(f"  Premium (bid): ${premium:.2f} ({premium_yield:.2f}% yield)")
                print(f"  If called: {upside + premium_yield:.1f}% total return")
        except Exception as e:
            pass

find_covered_call_opportunities('AAPL')

---

## Quiz

In [ ]:
quiz = [
    {"q": "What is the max loss on a long call?", "a": "B", 
     "opts": ["A) Unlimited", "B) The premium paid", 
              "C) The strike price", "D) The stock price"]},
    {"q": "A covered call requires?", "a": "C",
     "opts": ["A) Buying calls", "B) Selling naked calls", 
              "C) Owning stock + selling calls", "D) Buying puts"]},
    {"q": "Long call breakeven = ?", "a": "A",
     "opts": ["A) Strike + Premium", "B) Strike - Premium", 
              "C) Stock price + Premium", "D) Stock price - Premium"]},
    {"q": "Covered calls have what profit characteristic?", "a": "D",
     "opts": ["A) Unlimited profit", "B) Unlimited loss", 
              "C) No profit potential", "D) Capped profit at strike + premium"]},
    {"q": "Why are naked calls dangerous?", "a": "A",
     "opts": ["A) Unlimited loss potential if stock rises", "B) Limited profit", 
              "C) High premium cost", "D) They're not dangerous"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-B, 2-C, 3-A, 4-D, 5-A")
    
run_quiz()

---

## Summary

- **Long calls**: Bullish, limited loss, unlimited gain, pay premium
- **Breakeven** = Strike + Premium
- **Covered calls**: Own stock + sell calls for income
- **Never sell naked calls** as a beginner (unlimited risk)
- **Use OTM calls** for covered calls (5-10% above price)

**Tomorrow**: Day 18 - Put Options Deep Dive